In [56]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from wrangle import Wrangle
from prepare import Prepare
import pandas as pd
import numpy as np
from scipy import stats
train, val, test = Wrangle().wrangle_zillow()
train.head()

,bedrooms,bathrooms,square_feet,home_tax_value,year_built,fips,county
24082,3.0,2.0,1040.0,315000.0,1971.0,6037.0,Los Angeles
9432,4.0,1.0,1354.0,51513.0,1947.0,6037.0,Los Angeles
50142,3.0,1.0,1180.0,64709.0,1949.0,6037.0,Los Angeles
38310,3.0,2.0,1922.0,218663.0,1952.0,6037.0,Los Angeles
7652,4.0,4.0,3710.0,1700000.0,1987.0,6037.0,Los Angeles


In [6]:
p = Prepare()
(X_train, y_train), (X_val, y_val), (X_test, y_test) = p.get_Xy(train, val, test)
X_train.head()

,bedrooms,bathrooms,square_feet
24082,3.0,2.0,1040.0
9432,4.0,1.0,1354.0
50142,3.0,1.0,1180.0
38310,3.0,2.0,1922.0
7652,4.0,4.0,3710.0


In [27]:
X_train_scaled, X_val_scaled, X_test_scaled, _ = p.scaling(X_train, X_val, X_test)
X_train_scaled.head()

,bedrooms,bathrooms,square_feet
0,-0.304758,-0.258425,-1.018840
1,0.830145,-1.417729,-0.614800
2,-0.304758,-1.417729,-0.838695
3,-0.304758,-0.258425,0.116074
4,0.830145,2.060183,2.416785


In [8]:
baseline_pred = train["home_tax_value"].mean()
train['baseline_prediction'] = baseline_pred
train.head()

,bedrooms,bathrooms,square_feet,home_tax_value,year_built,fips,county,baseline_prediction
24082,3.0,2.0,1040.0,315000.0,1971.0,6037.0,Los Angeles,455123.241223
9432,4.0,1.0,1354.0,51513.0,1947.0,6037.0,Los Angeles,455123.241223
50142,3.0,1.0,1180.0,64709.0,1949.0,6037.0,Los Angeles,455123.241223
38310,3.0,2.0,1922.0,218663.0,1952.0,6037.0,Los Angeles,455123.241223
7652,4.0,4.0,3710.0,1700000.0,1987.0,6037.0,Los Angeles,455123.241223


In [9]:
baseline_rmse = mean_squared_error(train["home_tax_value"], train["baseline_prediction"], squared=False)
baseline_rmse

386069.73075485

### Baseline RMSE is 386069.73

In [10]:
def grid_search(X, y, model, params_dic):
    grid = GridSearchCV(model, params_dic, n_jobs=-1)
    return grid.fit(X, y)

In [22]:
lasso_grid = {"alpha": [0.01, 0.1, 0.25, 0.5, 0.75, 1.0]}
tweedie_grid = {"power": [0, 1, 2, 3], "alpha": [0.01, 0.1, 0.25, 0.5, 0.75, 1.0]}
models = [LassoLars(random_state=123, normalize=False), TweedieRegressor()]

lasso = grid_search(X_train_scaled, y_train, models[0], lasso_grid)
tweddie = grid_search(X_train_scaled, y_train, models[1], tweedie_grid)

In [47]:
pf = PolynomialFeatures(degree=4)
X_train_degree2 = pf.fit_transform(X_train)
X_validate_degree2 = pf.transform(X_val)
X_test_degree2 = pf.transform(X_test)

In [48]:
lm2 = LinearRegression()
lm2.fit(X_train_degree2, y_train)
lm2_predic_train = lm2.predict(X_train_degree2)
mean_squared_error(y_train, lm2_predic_train, squared=False)

316001.37456703477

In [23]:
lasso.best_estimator_


LassoLars(normalize=False, random_state=123)

In [24]:
lasso.best_params_

{'alpha': 1.0}

In [25]:
tweddie.best_estimator_

TweedieRegressor(power=1)

In [26]:
tweddie.best_params_

{'alpha': 1.0, 'power': 1}

In [57]:
lasso = LassoLars(alpha=0.75, normalize=False).fit(X_train_scaled, y_train)
tweedie = TweedieRegressor(power=1, alpha=0.75).fit(X_train_scaled, y_train)
lr = LinearRegression().fit(X_train_scaled, y_train)

In [58]:
lass_pred_train = lasso.predict(X_train_scaled)
tweed_pred_train = tweedie.predict(X_train_scaled)
lr_pred_train = lr.predict(X_train_scaled)

In [60]:
mean_squared_error(y_train, tweed_pred_train, squared=False)

319198.0150116717

In [19]:
mean_squared_error(X_train_scaled, lass_pred_train, squared=False)

318602.79264156776

In [20]:
mean_squared_error(X_train_scaled, lr_pred_train, squared=False)

318602.77199369104